In [1]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-384"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, use_fast=True)

/mnt/e/ITSSxLTA-Hackathon2025/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchvision.transforms import Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
# _transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])
_transforms = Compose([ToTensor(), normalize])

In [3]:
from data_stuff import CSVDataset, Transforms

tran = Transforms(
    image_processor,
    img_size=384,
)

ds_train  = CSVDataset('/mnt/e/data/dataset_train.csv', tran)
ds_val  = CSVDataset('/mnt/e/data/dataset_val.csv', tran)
ds_test  = CSVDataset('/mnt/e/data/dataset_test.csv', tran)

In [4]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

# labels = food["train"].features["label"].names
# label2id, id2label = dict(), dict()
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label
 
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    # num_labels=2,
)

In [5]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)



In [6]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="my_awesome_model_more_balanced",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    processing_class=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.231500,0.948308,0.595682
2,0.217600,1.208244,0.519512
3,0.201000,1.046589,0.539493
4,0.237500,1.218486,0.491216
5,0.280500,0.778194,0.631554
6,0.252400,1.199095,0.533526
7,0.190800,0.780303,0.601985
8,0.194800,1.031234,0.567386
9,0.113000,0.982225,0.522194
10,0.141500,1.168725,0.518573


KeyboardInterrupt: 

In [ ]:
from datasets import Dataset
from data_stuff import CSVDataset, Transforms

tran = Transforms(
    image_processor,
    img_size=384,
)

ds_train  = CSVDataset('/mnt/e/data/dataset_train.csv', tran)

samples_df = ds_test.df
print(samples_df['label'].value_counts())
test_ds_hf = Dataset.from_pandas(samples_df)
# print(test_ds_hf)

label
0    3173
1    1339
Name: count, dtype: int64


In [2]:
from evaluate import evaluator
from transformers import pipeline
from transformers import AutoImageProcessor
from transformers import AutoModelForImageClassification

checkpoint = "/mnt/e/ITSSxLTA-Hackathon2025/my_awesome_model_more_balanced/checkpoint-1915"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, use_fast=True)
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    # num_labels=2,
)

/mnt/e/ITSSxLTA-Hackathon2025/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
task_evaluator = evaluator("image-classification")
pipe = pipeline("image-classification", model=model, image_processor=image_processor)
eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=test_ds_hf,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1}
)

print(eval_results)

Device set to use cuda:0


label mapping {'LABEL_0': 0, 'LABEL_1': 1}
predictions [[{'label': 'tench, Tinca tinca', 'score': 0.5370111465454102}, {'label': 'goldfish, Carassius auratus', 'score': 0.4626675248146057}, {'label': 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi', 'score': 1.3565057088271715e-05}, {'label': 'traffic light, traffic signal, stoplight', 'score': 1.3370806300372351e-05}, {'label': 'street sign', 'score': 7.210293460957473e-06}], [{'label': 'goldfish, Carassius auratus', 'score': 0.521441638469696}, {'label': 'tench, Tinca tinca', 'score': 0.4782581329345703}, {'label': 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi', 'score': 1.2647704352275468e-05}, {'label': 'traffic light, traffic signal, stoplight', 'score': 1.2568131751322653e-05}, {'label': 'street sign', 'score': 6.839883553766413e-06}], [{'label': 'tench, Tinca tinca', 'score': 0.6234973073005676}, {'label': 'goldfish, Carassius auratus', 'score': 0.37623220682144165}, {'la

KeyError: 'tench, Tinca tinca'